# Data cleaning

In [1]:
import pandas as pd
import os


In [2]:
DATA_PATH = "../../DATA+RESULTS/"
SERVER_COLLECTION_1 = "PMS-SERVER-07-06-2019/"


path = DATA_PATH + SERVER_COLLECTION_1
rawFile = path + 'datarecord.csv'
aqiFile = path + 'partnerpreviousaqi.csv'
cleanFile = path + "cleanfile.csv"


In [18]:
# (No effect)
tempData = open(rawFile, "r").readlines()

for i in range(len(tempData)):
    tempData[i] = tempData[i].replace(";", ",").replace("\"", "")

open(rawFile, "w").writelines(tempData)

tempData = open(aqiFile, "r").readlines()
for i in range(len(tempData)):
    tempData[i] = tempData[i].replace(";", ",").replace("\"", "")
open(aqiFile, "w").writelines(tempData)

tempData = None


In [3]:
# Load csv file
rawTable = pd.read_csv(rawFile)
aqiTable = pd.read_csv(aqiFile)

# show data file
rawTable.head()

,id,aqi,device_id,humidity,pm1,pm10,pm25,temperature
0,133,129,1,41.2,30,62,47,24.8
1,136,129,1,41.1,30,62,47,24.8
2,139,136,1,41.1,31,66,50,24.8
3,142,136,1,41.1,32,67,50,24.8
4,145,136,1,41.1,32,68,50,24.8


In [15]:
# show aqi file
aqiTable.head()

,id,aqi,deviceid,timestamp
0,134,129,1,2019-03-13 21:59:59
1,137,129,1,2019-03-13 22:00:21
2,140,136,1,2019-03-13 22:00:43
3,143,136,1,2019-03-13 22:01:04
4,146,136,1,2019-03-13 22:01:26


In [4]:
# Decrement the id of AQI table 
# To make it equal to raw table

#   main reason behind this server first puts the
# the record in raw table then increments its id
# generator than puts the reocrd in AQI table

aqiTable["id"] = aqiTable["id"] - 1
aqiTable.head()

,id,aqi,deviceid,timestamp
0,133,129,1,2019-03-13 21:59:59
1,136,129,1,2019-03-13 22:00:21
2,139,136,1,2019-03-13 22:00:43
3,142,136,1,2019-03-13 22:01:04
4,145,136,1,2019-03-13 22:01:26


In [5]:
mergedDataTable = rawTable.merge(aqiTable, left_on='id', right_on='id', how='inner')
mergedDataTable.head()

,id,aqi_x,device_id,humidity,pm1,pm10,pm25,temperature,aqi_y,deviceid,timestamp
0,133,129,1,41.2,30,62,47,24.8,129,1,2019-03-13 21:59:59
1,136,129,1,41.1,30,62,47,24.8,129,1,2019-03-13 22:00:21
2,139,136,1,41.1,31,66,50,24.8,136,1,2019-03-13 22:00:43
3,142,136,1,41.1,32,67,50,24.8,136,1,2019-03-13 22:01:04
4,145,136,1,41.1,32,68,50,24.8,136,1,2019-03-13 22:01:26


In [7]:
#   If coresponding aqi is same in both tables
# than data is clean. No furthure processing will be required.

# get AQI column came from both tables
aqi_x = mergedDataTable["aqi_x"] # AQI came from "RAW" table
aqi_y = mergedDataTable["aqi_y"] # AQI came from "AQI" table

# get the data common in both tables
cleanDataTable = mergedDataTable[aqi_y == aqi_x]
cleanDataTable.head()

,id,aqi_x,device_id,humidity,pm1,pm10,pm25,temperature,aqi_y,deviceid,timestamp
0,133,129,1,41.2,30,62,47,24.8,129,1,2019-03-13 21:59:59
1,136,129,1,41.1,30,62,47,24.8,129,1,2019-03-13 22:00:21
2,139,136,1,41.1,31,66,50,24.8,136,1,2019-03-13 22:00:43
3,142,136,1,41.1,32,67,50,24.8,136,1,2019-03-13 22:01:04
4,145,136,1,41.1,32,68,50,24.8,136,1,2019-03-13 22:01:26


In [8]:
# number of rows in data
print('Number of rows in "RAW" data:', len(rawTable))
print('Number of rows in "AQI" data:', len(aqiTable))
print('Number of rows in "MERRGED" data:', len(mergedDataTable))
print('Number of rows in "CLEAN" data:', len(cleanDataTable))

Number of rows in "RAW" data: 707004
Number of rows in "AQI" data: 707004
Number of rows in "MERRGED" data: 706998
Number of rows in "CLEAN" data: 706998


In [9]:
cleanDataTable.rename(columns={"aqi_x":"aqi"}, inplace=True)
cleanDataTable = cleanDataTable[["timestamp", "device_id", "aqi", 
                                 "temperature", "humidity", "pm1", "pm25", "pm10"]]
cleanDataTable.head()

,timestamp,device_id,aqi,temperature,humidity,pm1,pm25,pm10
0,2019-03-13 21:59:59,1,129,24.8,41.2,30,47,62
1,2019-03-13 22:00:21,1,129,24.8,41.1,30,47,62
2,2019-03-13 22:00:43,1,136,24.8,41.1,31,50,66
3,2019-03-13 22:01:04,1,136,24.8,41.1,32,50,67
4,2019-03-13 22:01:26,1,136,24.8,41.1,32,50,68


In [10]:
# (Found this error while plotting data)

# a number of values are corrput
# due to server error or device error
cleanDataTable[cleanDataTable['pm25'] > 1000].head()

# temperature and humidity is okay
# so problem is not from the server side
# it is probably from device side
# specifically from PMS sensor reading

,timestamp,device_id,aqi,temperature,humidity,pm1,pm25,pm10
15027,2019-03-15 02:06:22,1,0,24.8,45.9,65535,65535,65535
18088,2019-03-15 03:52:24,1,0,24.7,46.2,65535,65535,65535
34702,2019-03-15 14:29:48,1,0,24.7,44.0,65535,65535,65535
38953,2019-03-15 17:50:55,1,0,25.2,44.0,65535,65535,65535
53860,2019-03-16 02:43:56,1,0,24.5,45.7,65535,65535,65535


In [11]:
cleanDataTable = cleanDataTable[cleanDataTable['pm25'] != 65535]


In [13]:
cleanDataTable.to_csv(cleanFile)